# Inference in a Propositional Knowledge Base

### Import Modules

In [1]:
from utils import *
from logic import *
from notebook import psource

## Wumpus World KB
Let us create a `PropKB` for the wumpus world with the sentences 

We define the symbols we use in our clauses.<br/>
$P_{x, y}$ is true if there is a pit in `[x, y]`.<br/>
$B_{x, y}$ is true if the agent senses breeze in `[x, y]`.<br/>

In [2]:
P11, P12, P21, P22, P31, B11, B21 = expr('P11, P12, P21, P22, P31, B11, B21')

There is no pit in `[1,1]`.

In [3]:
wumpus_kb.tell(~P11)

A square is breezy if and only if there is a pit in a neighboring square. This has to be stated for each square but for now, we include just the relevant squares.

In [4]:
wumpus_kb.tell(B11 | '<=>' | ((P12 | P21)))
wumpus_kb.tell(B21 | '<=>' | ((P11 | P22 | P31)))

Now we include the breeze percepts for the first two squares leading up to the situation in 

In [5]:
wumpus_kb.tell(~B11)
wumpus_kb.tell(B21)

We can check the clauses stored in a `KB` by accessing its `clauses` variable

In [6]:
wumpus_kb.clauses

[~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21,
 ~P11,
 (~P12 | B11),
 (~P21 | B11),
 (P12 | P21 | ~B11),
 (~P11 | B21),
 (~P22 | B21),
 (~P31 | B21),
 (P11 | P22 | P31 | ~B21),
 ~B11,
 B21]

We see that some equivalences  were automatically converted to two implications which were inturn converted to CNF which is stored in the `KB`.<br/>

### Truth Table Enumeration
It is a model-checking approach which, as the name suggests, enumerates all possible models in which the `KB` is true and checks if $\alpha$ is also true in these models. We list the $n$ symbols in the `KB` and enumerate the $2^{n}$ models in a depth-first manner and check the truth of `KB` and $\alpha$.

In [7]:
psource(tt_check_all)

The algorithm basically computes every line of the truth table $KB\implies \alpha$ and checks if it is true everywhere.
<br>

`tt_entails()` just extracts the symbols from the query and calls `tt_check_all()` with the proper parameters.


In [8]:
psource(tt_entails)

Keep in mind that for two symbols P and Q, P => Q is false only when P is `True` and Q is `False`.
Example usage of `tt_entails()`:

In [9]:
tt_entails(P & Q, Q)

True

P & Q is True only when both P and Q are True. Hence, (P & Q) => Q is True

In [10]:
tt_entails(P | Q, Q)

False

In [11]:
tt_entails(P | Q, P)

False

If we know that P | Q is true, we cannot infer the truth values of P and Q. 
Hence (P | Q) => Q is False and so is (P | Q) => P.

In [12]:
(A, B, C, D, E, F, G) = symbols('A, B, C, D, E, F, G')
tt_entails(A & (B | C) & D & E & ~(F | G), A & D & E & ~F & ~G)

True

We can see that for the KB to be true, A, D, E have to be True and F and G have to be False.
Nothing can be said about B or C.

Coming back to our problem, note that `tt_entails()` takes an `Expr` which is a conjunction of clauses as the input instead of the `KB` itself. 
You can use the `ask_if_true()` method of `PropKB` which does all the required conversions. 
Let's check what `wumpus_kb` tells us about $P_{1, 1}$.

In [13]:
wumpus_kb.ask_if_true(~P11), wumpus_kb.ask_if_true(P11)

(True, False)

### Proof by Resolution


The algorithm that implements proof by resolution cannot handle complex sentences. Implications and bi-implications have to be simplified into simpler clauses. We already know that every sentence of a propositional logic is logically equivalent to a conjunction of clauses. We will use this fact to our advantage and simplify the input sentence into the conjunctive normal form (CNF) which is a conjunction of disjunctions of literals
The `to_cnf` function executes this conversion using helper subroutines.

In [14]:
psource(to_cnf)

Let's convert some sentences to see how it works


In [15]:
A, B, C, D = expr('A, B, C, D')
to_cnf(A |'<=>'| B)

((A | ~B) & (B | ~A))

In [16]:
to_cnf(A |'<=>'| (B & C))

((A | ~B | ~C) & (B | ~A) & (C | ~A))

In [17]:
to_cnf(A & (B | (C & D)))

(A & (C | B) & (D | B))

In [18]:
to_cnf((A |'<=>'| ~B) |'==>'| (C | ~D))

((B | ~A | C | ~D) & (A | ~A | C | ~D) & (B | ~B | C | ~D) & (A | ~B | C | ~D))

Coming back to our resolution problem, we can see how the `to_cnf` function is utilized here

In [19]:
psource(pl_resolution)

In [20]:
pl_resolution(wumpus_kb, ~P22), pl_resolution(wumpus_kb, P22)

(False, False)

In [21]:
pl_resolution(wumpus_kb, ~P11), pl_resolution(wumpus_kb, P11)

(True, False)